In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from fast_ml.model_development import train_valid_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Подготовка-выборок-для-моделей" data-toc-modified-id="Подготовка-выборок-для-моделей-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка выборок для моделей</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

### Предобработка данных

In [6]:
df = pd.read_csv('Churn.csv')

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


С **типами данных** все в порядке.  
Обнаружены **пустые значения** в столбце Tenure. Можно заполнить их медианным значением.

In [9]:
df.loc[df['Tenure'].isna(), 'Tenure'] = df['Tenure'].median()

**Проверим на дубли**

In [10]:
df[df.duplicated()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


In [11]:
df[df.RowNumber.duplicated()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


In [12]:
df[df.CustomerId.duplicated()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


**Вывод:**
- Дублей не обнаружено
- типы столбцов корректные
- заполнены пропуски
- данные можно использовать

### Подготовка выборок для моделей

Такие данные, как фамилия клиента, номер строки, ID клиента, моделям ни о чем не скажут.

In [13]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

Категориальные текстовые данные кодируем методом OHE

In [14]:
df = pd.get_dummies(df, drop_first=True)

Разделяем наши данные на 3 выборки

In [15]:
features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(
    df, target = 'Exited', train_size=0.7, valid_size=0.15, test_size=0.15, random_state=999)

print('train:', features_train.shape)
print('valid:', features_valid.shape)
print('test:', features_test.shape)

train: (7000, 11)
valid: (1500, 11)
test: (1500, 11)


In [16]:
# колонки для масштабирования
numerics = ['int64', 'float64', 'uint8']
numeric_cols = df.select_dtypes(include=numerics).columns.tolist()
numeric_cols.remove('Exited')
numeric_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Geography_Germany',
 'Geography_Spain',
 'Gender_Male']

In [17]:
# # объявляем scaler
scaler = StandardScaler()
# обучаем
scaler.fit(features_train[numeric_cols])
# Масштабируем
features_train[numeric_cols] = scaler.transform(features_train[numeric_cols])
features_valid[numeric_cols] = scaler.transform(features_valid[numeric_cols])
features_test[numeric_cols] = scaler.transform(features_test[numeric_cols])

In [18]:
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2572,0.620491,1.853015,1.381546,0.227444,0.801800,0.648202,0.969882,-1.285115,-0.578230,-0.570532,0.918720
4540,-0.676503,-0.655623,-1.378391,1.168705,-0.911448,0.648202,-1.031053,-1.196694,1.729416,-0.570532,0.918720
8384,1.077033,-0.462651,-0.688407,-1.240591,0.801800,0.648202,0.969882,1.021341,-0.578230,-0.570532,0.918720
3149,-0.085074,-0.462651,1.036554,0.648247,-0.911448,0.648202,-1.031053,-0.610775,-0.578230,1.752751,-1.088471
5479,-0.852895,-0.366165,-0.688407,0.640494,-0.911448,-1.542728,-1.031053,-0.102248,-0.578230,1.752751,0.918720


## Исследование задачи

Посмотрим на наличие дисбаланса классов

In [19]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

**Вывод**: Дисбаланс определенно присутствует

Попробуем обучить модели с без учета дисбаланса классов и посчитать метрику F1

In [20]:
# DecisionTreeClassifier
best_f1 = 0
best_depth = 0
for depth in range(2, 17):
    model = DecisionTreeClassifier(random_state=999, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth

print('Best depth:', best_depth, 'F1:', best_f1)

Best depth: 5 F1: 0.5962264150943397


In [21]:
# RandomForestClassifier
# сначала подбираем лучшую глубину
best_f1 = 0
best_depth = 0
for depth in range(2, 17):
    model = RandomForestClassifier(random_state=999, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth

# по лучшей глубине подберем лучшее число деревьев
best_estims = 100 #by default
for estims in range(20, 121, 10):
    model = RandomForestClassifier(random_state=999, max_depth=best_depth, n_estimators=estims)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_estims = estims
        
print('Best depth:', best_depth, 'Best estims', best_estims, 'F1:', best_f1)

Best depth: 14 Best estims 100 F1: 0.6012024048096192


In [22]:
# LogisticRegression
# подберем лучшее число итераций обучения
best_iters = 0 
best_f1 = 0
for iters in range(1, 15, 1):
    model = LogisticRegression(max_iter=iters, random_state=999, solver='lbfgs')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_iters = iters
        
print('Best iters', best_iters, 'F1:', best_f1)

Best iters 3 F1: 0.33185840707964603


**Вывод:**  
Лучшая модель без учета дибаланса классов - RandomForestClassifier с лучшим найденным `F1 = 0.6`

## Борьба с дисбалансом

Попробуем использовать гиперпараметр `class_weight='balanced'`

In [26]:
# DecisionTreeClassifier
best_f1 = 0
best_depth = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=999, max_depth=depth, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth

print('Best depth:', best_depth, 'F1:', best_f1)

Best depth: 7 F1: 0.5794621026894865


Для решающего дерева результат почему-то стал хуже, чем был

In [27]:
# RandomForestClassifier
# сначала подбираем лучшую глубину
best_f1 = 0
best_depth = 0
for depth in range(1, 16):
    model = RandomForestClassifier(random_state=999, max_depth=depth, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth

# по лучшей глубине подберем лучшее число деревьев
best_estims = 100 #by default
for estims in range(10, 161, 10):
    model = RandomForestClassifier(random_state=999, max_depth=best_depth, n_estimators=estims, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_estims = estims
        
print('Best depth:', best_depth, 'Best estims', best_estims, 'F1:', best_f1)

Best depth: 9 Best estims 120 F1: 0.6505295007564296


RandomForestClassifier: F1 составил 0.65, что уже лучше против 0.6 для модели без учета несбалансированности классов

In [22]:
# LogisticRegression
# подберем лучшее число итераций обучения
best_iters = 0 
best_f1 = 0
for iters in range(1, 20, 1):
    model = LogisticRegression(max_iter=iters, random_state=999, solver='lbfgs', class_weight='balanced')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_iters = iters
        
print('Best iters', best_iters, 'F1:', best_f1)

Best iters 4 F1: 0.5147392290249434


LogisticRegression: результат стал значительно лучше.

Теперь сравним применение гиперпараметра `class_weight='balanced'` с **upsampling**'ом

In [23]:
def upsample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # рассчитаем соотношение классов
    repeat = round(len(target_zeros) / len(target_ones))
        
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=999)
    
    return features_upsampled, target_upsampled

In [24]:
features_train_upsampld, features_target_upsampld = upsample(features_train, target_train)

In [25]:
# проверяем баланс классов
features_target_upsampld.value_counts()

1    5788
0    5553
Name: Exited, dtype: int64

Теперь мы имеем примерно поровну значений в каждом классе. Обучим модель:

In [26]:
# RandomForestClassifier
# сначала подбираем лучшую глубину
best_f1 = 0
best_depth = 0
for depth in range(1, 16):
    model = RandomForestClassifier(random_state=999, max_depth=depth)
    model.fit(features_train_upsampld, features_target_upsampld)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth

# по лучшей глубине подберем лучшее число деревьев
best_estims = 100 #by default
for estims in range(10, 161, 10):
    model = RandomForestClassifier(random_state=999, max_depth=best_depth, n_estimators=estims)
    model.fit(features_train_upsampld, features_target_upsampld)
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_estims = estims
        
print('Best depth:', best_depth, 'Best estims', best_estims, 'F1:', best_f1)

Best depth: 11 Best estims 70 F1: 0.6397608370702541


Upsampling показал худшие результаты (0.64) против class_weight='balanced' (0.652)

**Вывод:**
- Лучший F1 показала модель `RandomForestClassifier`
- Были подобраны следующие лучшие гиперпараметры: `depth: 9`, `n_estimators: 120`, `class_weight='balanced'`
- Метрика **F1: 0.652** (на валидационной выборке)
- С несбалансированностью классов лучше помог справиться гиперпараметр `class_weight='balanced'`, нежели `upsampling` обучающих выборок.

## Тестирование модели

Заново обучим нашу лучшую модель, т.к. ранее она "перезаписалась"

In [28]:
model = RandomForestClassifier(random_state=999, max_depth=9, n_estimators=120, class_weight='balanced')
model.fit(features_train, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=120,
                       random_state=999)

Проверим предсказания на тестовой выборке

In [29]:
predictions = model.predict(features_test)
f1 = f1_score(target_test, predictions)

In [30]:
f1

0.6160990712074303

Посчитаем **ROC-AUC**

In [31]:
prob_test = model.predict_proba(features_test)
prob_test = prob_test[:, 1]

auc_roc = roc_auc_score(target_test, prob_test)
print(auc_roc)

0.8600880802830121


**Финальный вывод:**
- Данные были открыты, предобработаны, проверены
- Подготовлены обучающая, валидационная и тестовая выборки с примененим техник one-hot encoding, переменные отмасштабированы
- Исследован баланс классов. Выявлен дисбаланс. Модели были обучены с учетом дисбаланса и без. Проведено сравнение работы гиперпараметра `class_weight='balanced'` и техники `upsampling`. Гиперпараметр оказался эффективнее.
- Выяснено, что лучшая модель по метрике F1 score - Это `RandomForestClassifier`. Для неё перебором подобраны наилучшие гиперпараметры.
- **F1** модели на **тестовой** выборке составил `0.618`. **ROC-AUC** score = `0.86`, что говорит о том, что модель предсказывает лучше, чем случайные числа.